<a href="https://colab.research.google.com/github/fagonzalezo/qmc/blob/master/examples/QMClassiffierSGD_EigDec_vs_SDec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install qmc if running in Google Colab

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install git+https://github.com/fagonzalezo/qmc.git
else:
    import sys
    sys.path.insert(0, "../")

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import numpy as np
import tensorflow as tf
import qmc.tf.layers as layers
import qmc.tf.models as models


# Comparison of `QMeasureClassifEig` and `QMClassifSDecompFDMatrix`

QMeasureClassifEig uses a decomposition of the rho density matrix analogous to an 
Eigen decomposition. The number of parameters is $dim_x \times dim_y \times n_{comp}$. QMClassifSDecompFDMatrix uses a decomposition analogous to a
Schmidt decomposition. The number of parameters is $(dim_x + dim_y) \times n_{comp}$. We build two models with similar number of parameters. 

In [3]:
X,y = load_digits(return_X_y=True)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, stratify=y, random_state=42)

print(X_train.shape)
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
y_train_bin = tf.reshape (tf.keras.backend.one_hot(y_train, 10), (-1,10))
y_test_bin = tf.reshape (tf.keras.backend.one_hot(y_test, 10), (-1,10))
print(y_train.shape)
print(y_train_bin.shape)

(1203, 64)
(1203, 1)
(1203, 10)


In [4]:
fm_x = layers.QFeatureMapRFF(64, dim=150, gamma=0.375, random_state=17)
fm_y = layers.QFeatureMapOneHot(num_classes=10)
qmc_dig = models.QMClassifier(fm_x=fm_x, fm_y=fm_y, dim_x=150, dim_y=10)
qmc_dig.compile()
qmc_dig.fit(X_train, y_train, epochs=1)
out = qmc_dig.predict(X_test)
print(accuracy_score(y_test, np.argmax(out, axis=1)))

38/38 [==============================] - 3s 79ms/step - loss: 0.0000e+00
0.9377104377104377


In [5]:
qmc_sgd_eig = models.QMClassifierSGD(input_dim=64, dim_x=150, dim_y=10,
                                     num_eig=10, gamma=0.375, random_state=17)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
qmc_sgd_eig.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy())
qmc_sgd_eig.fit(X_train, y_train_bin, epochs=10)
out = qmc_sgd_eig.predict(X_test)
print(accuracy_score(y_test, np.argmax(out, axis=1)))

Epoch 1/10
38/38 [==============================] - 0s 1ms/step - loss: 1.3154
Epoch 2/10
38/38 [==============================] - 0s 1ms/step - loss: 0.3695
Epoch 3/10
38/38 [==============================] - 0s 1ms/step - loss: 0.2446
Epoch 4/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1995
Epoch 5/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1526
Epoch 6/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1281
Epoch 7/10
38/38 [==============================] - 0s 2ms/step - loss: 0.1135
Epoch 8/10
38/38 [==============================] - 0s 1ms/step - loss: 0.0979
Epoch 9/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0853
Epoch 10/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0815
0.9696969696969697


In [6]:
qmc_sgd_eig.summary()

Model: "qm_classifier_sgd"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
q_feature_map_rff_1 (QFeatur multiple                  9750      
_________________________________________________________________
q_measure_classif_eig (QMeas multiple                  15010     
_________________________________________________________________
density_matrix2dist_1 (Densi multiple                  0         
Total params: 24,760
Trainable params: 24,760
Non-trainable params: 0
_________________________________________________________________


In [7]:
inputs = tf.keras.Input(shape=(64,))
fm_x = layers.QFeatureMapRFF(64, dim=150, gamma=0.375, random_state=17)
psi_x = fm_x(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmc_sd_fdm = layers.QMClassifSDecompFDMatrix(dim_x=150, dim_y=10, n_comp=90)
rho_y = qmc_sd_fdm(rho_x)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmc_sgd_sdec = tf.keras.Model(inputs=inputs, outputs=probs)

In [8]:
qmc_sgd_sdec.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy())
qmc_sgd_sdec.fit(X_train, y_train_bin, epochs=10)
out = qmc_sgd_sdec.predict(X_test)
print(accuracy_score(y_test, np.argmax(out, axis=1)))

Epoch 1/10
38/38 [==============================] - 1s 5ms/step - loss: 1.2970
Epoch 2/10
38/38 [==============================] - 0s 6ms/step - loss: 0.3042
Epoch 3/10
38/38 [==============================] - 0s 6ms/step - loss: 0.1593
Epoch 4/10
38/38 [==============================] - 0s 6ms/step - loss: 0.1174
Epoch 5/10
38/38 [==============================] - 0s 6ms/step - loss: 0.0939
Epoch 6/10
38/38 [==============================] - 0s 5ms/step - loss: 0.0789
Epoch 7/10
38/38 [==============================] - 0s 5ms/step - loss: 0.0698
Epoch 8/10
38/38 [==============================] - 0s 5ms/step - loss: 0.0643
Epoch 9/10
38/38 [==============================] - 0s 5ms/step - loss: 0.0565
Epoch 10/10
38/38 [==============================] - 0s 5ms/step - loss: 0.0527
0.9781144781144782


In [9]:
qmc_sgd_sdec.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 1)            0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.ones_like (TFOpLambda)       (None, 1)            0           tf.__operators__.getitem[0][0]   
__________________________________________________________________________________________________
q_feature_map_rff_2 (QFeatureMa (None, 150)          9750        input_1[0][0]                    
______________________________________________________________________________________________